In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# --- 1. Define Constants and Paths ---
DATA_DIR = 'data/Plant_leave_diseases_dataset_without_augmentation'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# --- 2. Load the Data from the Directory ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# --- 3. Explore the Data ---
class_names = train_dataset.class_names
print(f"Found the following classes: {class_names}")
print(f"Number of classes: {len(class_names)}")

# Let's look at the shape of our data batches
for images, labels in train_dataset.take(1):
    print(f"\nImages batch shape: {images.shape}")
    print(f"Labels batch shape: {labels.shape}")
    

Found 55448 files belonging to 39 classes.
Using 44359 files for training.
Found 55448 files belonging to 39 classes.
Using 11089 files for validation.
Found the following classes: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Background_without_leaves', 'Blueberry___healthy', 'Cherry___Powdery_mildew', 'Cherry___healthy', 'Corn___Cercospora_leaf_spot Gray_leaf_spot', 'Corn___Common_rust', 'Corn___Northern_Leaf_Blight', 'Corn___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'T

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os

# --- 1. Define Constants and Paths ---
DATA_DIR = 'data/Plant_leave_diseases_dataset_without_augmentation'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 8

# --- 2. Load the Data from the Directory ---
train_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

# --- 3. Explore the Data ---
class_names = train_dataset.class_names
print(f"Found the following classes: {class_names}")
print(f"Number of classes: {len(class_names)}")

# Let's look at the shape of our data batches
for images, labels in train_dataset.take(1):
    print(f"\nImages batch shape: {images.shape}")
    print(f"Labels batch shape: {labels.shape}")

Found 55448 files belonging to 39 classes.
Using 44359 files for training.


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(8):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
# --- Configure the dataset for performance ---
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# --- Create the data augmentation layers ---
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal", input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
  ]
)

In [ ]:
# --- Load the pre-trained base model ---
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Let's look at the model's architecture
base_model.summary()

In [ ]:
# --- Get the number of classes ---
num_classes = len(class_names)

# --- Build the final model by stacking layers ---
model = tf.keras.Sequential([
  data_augmentation,
  base_model,
  # Add our new classification layers on top
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dropout(0.2), # Helps prevent overfitting
  tf.keras.layers.Dense(num_classes) # The final output layer
])

# --- Compile the model ---
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# --- See the final model architecture ---
model.summary()

In [ ]:
# --- Train the model ---
epochs = 10
history = model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=epochs
)